In [1]:
import requests
import json
import pandas as pd
import datetime
import sqlite3

## Get Site Info from API

In [2]:
key1 = 'cUEwsckBq7bZ0wWCZL4jYMIGQicGUKd7'
site = 'https://api.meh.com/1/current.json?apikey=' + key1

In [3]:
current_dt = datetime.datetime.now()
current_date = str(current_dt.date())
current_time = str(current_dt.time())
r = requests.get(site)
status_code = r.status_code
print(current_dt)

2020-01-23 00:32:45.207413


In [4]:
assert(status_code == 200), "Status code != 200, Status code: "+str(status_code)

In [5]:
site_data = r.json()

## Save Raw Response 

In [27]:
db_location = '../data/meh_scraper_qa.db'

In [28]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [29]:
raw_data_series = pd.Series([json.dumps(site_data)], index=[current_date], name='raw_response')
raw_data_series.index.name = 'date'

In [30]:
raw_data_series.to_sql('raw_response_backup', con, if_exists='append', index=True)

In [31]:
con.commit()

## Begin Parsing JSON

In [32]:
def flatten_dict(parsed_site_series, k1):
    final_dict = {}
    for k,v in parsed_site_series.items():
        if k1 !='':
            new_k = k1 + '_' + k
        else:
            new_k = k

        if type(v) in [str, int, float]:
            final_dict[new_k] = v
        elif type(v) == dict:
            temp = flatten_dict(v, new_k)
            final_dict.update(temp)
        else:
            print(k + ' is ' + str(type(v)))
            pass
    return(final_dict)            

In [33]:
site_data_dict = flatten_dict(site_data, '')            
site_data_dict['date'] = current_date
site_data_dict['time'] = current_time
site_data_dict.keys()

items is <class 'list'>
photos is <class 'list'>
launches is <class 'list'>
answers is <class 'list'>


dict_keys(['deal_features', 'deal_id', 'deal_purchaseQuantity_maximumLimit', 'deal_purchaseQuantity_minimumLimit', 'deal_title', 'deal_specifications', 'deal_story_title', 'deal_story_body', 'deal_theme_accentColor', 'deal_theme_backgroundColor', 'deal_theme_backgroundImage', 'deal_theme_foreground', 'deal_url', 'poll_id', 'poll_startDate', 'poll_title', 'video_id', 'video_startDate', 'video_title', 'video_url', 'video_topic_commentCount', 'video_topic_createdAt', 'video_topic_id', 'video_topic_replyCount', 'video_topic_url', 'video_topic_voteCount', 'date', 'time'])

In [34]:
# print(json.dumps(site_data_dict, indent=4))

In [35]:
parsed_site_series = pd.DataFrame(site_data_dict, index=[1])

In [36]:
parsed_site_series.to_sql('products', con, if_exists='append', index=False)

In [37]:
con.commit()
con.close()

## TODOs  

* logic for items, photos, launches, answers  
* Special page scanner to record info on bottom of page multiple times during the day, maybe include topics, votes and other info more frequently as well
* Links for external and tags accoringly 